In [ ]:
import json

system_file_path = './data/system/test.3.txt'

system_aoa = {}

with open(system_file_path, 'r') as file:
    for line in file:
        decoded_line = json.loads(line)

        timestamp = decoded_line['timestamp']
        serial_num = decoded_line['serial_num']
        aoa = decoded_line['aoa']
        ss = decoded_line['ss']

        if timestamp in system_aoa:
            system_aoa[timestamp].update({serial_num: {'aoa': aoa, 'ss': ss}})
        else:
            system_aoa[timestamp] = {serial_num: {'aoa': aoa, 'ss': ss}}

system_aoa

In [ ]:
import numpy as np
import matplotlib.pyplot as plt


def intersection_point_calculation(angle1, angle2, distance):
    # Convert angles to radians
    angle1_rad = np.radians(angle1)
    angle2_rad = np.radians(angle2)
    
    # Calculate x-coordinate of the second endpoint (which lies on the y-axis)
    end_point2_x = 0
    
    # Calculate y-coordinate of the second endpoint
    end_point2_y = distance
    
    # Calculate direction vectors of the rays
    direction1 = np.array([np.cos(angle1_rad), np.sin(angle1_rad)])
    direction2 = np.array([np.cos(angle2_rad), np.sin(angle2_rad)])

    # Compute slopes
    m1 = direction1[1] / direction1[0]
    m2 = direction2[1] / direction2[0]

    A = np.array([[m1, -1], [m2, -1]])
    b = np.array([0 - m1 * 0, distance - m2 * 0])
    intersection = np.linalg.solve(A, b)
    
    # Calculate intersection point using vectorized calculations
    # If determinant of direction vectors is zero, rays are parallel
    det = np.linalg.det(np.stack([direction1, direction2], axis=0))
    if np.isclose(det, 0):
        print("The rays are parallel, no intersection point exists.")
        return None
    else:
        # Calculate intersection point using vectorized formula
        intersection = np.linalg.solve(np.stack([direction1, -direction2], axis=1), end_point2_x - np.array([0, end_point2_y]))
        return intersection
    
def magnitude(x, y):
    return np.sqrt(x*x + y*y)
    
ANTENNA_DISTANCE = 4.5212

i =0

xs=[]
ys=[]

for timestamp in list(system_aoa.keys()):
    if system_aoa[timestamp]['316405C']['ss'] > 0:
        [x,y] = intersection_point_calculation(system_aoa[timestamp]['3164076']['aoa'], system_aoa[timestamp]['316405C']['aoa'], ANTENNA_DISTANCE)
        print(x,y)
        if (x>3 or y> 3) and x<5:
            xs.append(x)
            ys.append(y)

print(xs)
plt.scatter(xs,ys)
plt.xlabel("X (Meters)")
plt.ylabel("Y (Meters)")

plt.title("System Output")

plt.grid(True)

# Show plot
plt.show()

In [ ]:
import pandas as pd
import gpxpy
import gpxpy.gpx

gpx_file = open('./data/gps/test.1.gpx')

gpx = gpxpy.parse(gpx_file)

gps_lat_long = []
gps_time = []

for track in gpx.tracks:
    for segment in track.segments:
        for point in segment.points:
            gps_lat_long.append([point.latitude, point.longitude])
            gps_time.append(point.time)

gps_frame = pd.DataFrame(gps_lat_long, index=gps_time, columns="latitude,longitude".split(','))

gps_frame
